In [1]:
!apt-get -qq -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1 > /dev/null
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!pip -q install https://github.com/kmaehashi/chainer-colab/releases/download/2018-02-06/cupy_cuda80-4.0.0b3-cp36-cp36m-linux_x86_64.whl
!pip -q install 'chainer==4.0.0b3'
!apt-get -qq -y install xvfb freeglut3-dev ffmpeg> /dev/null
!pip -q install chainerrl
!pip -q install gym
!pip -q install pyglet
!pip -q install pyopengl
!pip -q install pyvirtualdisplay
!pip install scikit-image

Extracting templates from packages: 100%


In [16]:
from sklearn import datasets, model_selection, svm, metrics
from sklearn.cross_validation import train_test_split
import numpy as np
mnist = datasets.fetch_mldata('MNIST original', data_home='./')
print(mnist.data.shape)
dataset = np.asarray(mnist.data, np.float32)
dataset = dataset /255
target = np.asarray(mnist.target, np.int32)
data_train, data_test, label_train, label_test = train_test_split(dataset, target, test_size=0.2)
data_train = data_train.reshape((len(data_train), 1, 28, 28))
data_test = data_test.reshape((len(data_test), 1, 28, 28))

x_train = []
x_test = []
for (train, label) in zip(data_train, label_train):
  x_train.append((train,label))
for (test, label) in zip(data_test, label_test):
  x_test.append((test,label))

# print(x_train[0][0].shape)
# print(x_train[0][1])

(70000, 784)


In [4]:
from __future__ import print_function
import os
import argparse
from os.path import join
import numpy as np
from os import listdir
from os.path import join
from scipy.misc import imread, imresize, imsave

import chainer
import chainer.links as L
from chainer import cuda
import chainer.functions as F
from chainer import training
from chainer.training import extensions
from chainer import serializers
from chainer.utils import force_array
from chainer import optimizers, cuda, serializers
from chainer import Variable
from chainer.dataset import dataset_mixin

/usr/local/lib/python3.6/dist-packages/cupy/core/fusion.py:659: FutureWarning: cupy.core.fusion is experimental. The interface can change in the future.
  util.experimental('cupy.core.fusion')


In [0]:
n_hidden = 100
epoch = 1


In [0]:
def add_noise(h, sigma=0.2):
    xp = cuda.get_array_module(h.data)
    if chainer.config.train:
        return h + sigma * xp.random.randn(*h.shape)
    else:
        return h

class Generator(chainer.Chain):

    def __init__(self, n_hidden, bottom_width=4, ch=512, wscale=0.02):
        super(Generator, self).__init__()
        self.n_hidden = n_hidden
        self.ch = ch
        self.bottom_width = bottom_width

        with self.init_scope():
            w = chainer.initializers.Normal(wscale)
            self.l0 = L.Linear(self.n_hidden, bottom_width * bottom_width * ch, initialW=w)
            self.dc1 = L.Deconvolution2D(ch, ch // 2, 4, 2, 1, initialW=w)
            self.dc2 = L.Deconvolution2D(ch // 2, ch // 4, 4, 2, 1, initialW=w)
            self.dc3 = L.Deconvolution2D(ch // 4, ch // 8, 4, 2, 1, initialW=w)
            self.dc4 = L.Deconvolution2D(ch // 8, 3, 3, 1, 1, initialW=w)
            self.bn0 = L.BatchNormalization(bottom_width * bottom_width * ch)
            self.bn1 = L.BatchNormalization(ch // 2)
            self.bn2 = L.BatchNormalization(ch // 4)
            self.bn3 = L.BatchNormalization(ch // 8)

    def make_hidden(self, batchsize):
        return np.random.uniform(-1, 1, (batchsize, self.n_hidden, 1, 1)).astype(np.float32)

    def __call__(self, z):
        h = F.reshape(F.relu(self.bn0(self.l0(z))), (len(z), self.ch, self.bottom_width, self.bottom_width))
        h = F.relu(self.bn1(self.dc1(h)))
        h = F.relu(self.bn2(self.dc2(h)))
        h = F.relu(self.bn3(self.dc3(h)))
        x = F.sigmoid(self.dc4(h))
        return x

class Discriminator(chainer.Chain):

    def __init__(self, bottom_width=4, ch=512, wscale=0.02):
        w = chainer.initializers.Normal(wscale)
        super(Discriminator, self).__init__()
        with self.init_scope():
            self.c0_0 = L.Convolution2D(3, ch // 8, 3, 1, 1, initialW=w)
            self.c0_1 = L.Convolution2D(ch // 8, ch // 4, 4, 2, 1, initialW=w)
            self.c1_0 = L.Convolution2D(ch // 4, ch // 4, 3, 1, 1, initialW=w)
            self.c1_1 = L.Convolution2D(ch // 4, ch // 2, 4, 2, 1, initialW=w)
            self.c2_0 = L.Convolution2D(ch // 2, ch // 2, 3, 1, 1, initialW=w)
            self.c2_1 = L.Convolution2D(ch // 2, ch // 1, 4, 2, 1, initialW=w)
            self.c3_0 = L.Convolution2D(ch // 1, ch // 1, 3, 1, 1, initialW=w)
            self.l4 = L.Linear(bottom_width * bottom_width * ch, 1, initialW=w)
            self.bn0_1 = L.BatchNormalization(ch // 4, use_gamma=False)
            self.bn1_0 = L.BatchNormalization(ch // 4, use_gamma=False)
            self.bn1_1 = L.BatchNormalization(ch // 2, use_gamma=False)
            self.bn2_0 = L.BatchNormalization(ch // 2, use_gamma=False)
            self.bn2_1 = L.BatchNormalization(ch // 1, use_gamma=False)
            self.bn3_0 = L.BatchNormalization(ch // 1, use_gamma=False)

    def __call__(self, x):
        h = add_noise(x)
        h = F.leaky_relu(add_noise(self.c0_0(h)))
        h = F.leaky_relu(add_noise(self.bn0_1(self.c0_1(h))))
        h = F.leaky_relu(add_noise(self.bn1_0(self.c1_0(h))))
        h = F.leaky_relu(add_noise(self.bn1_1(self.c1_1(h))))
        h = F.leaky_relu(add_noise(self.bn2_0(self.c2_0(h))))
        h = F.leaky_relu(add_noise(self.bn2_1(self.c2_1(h))))
        h = F.leaky_relu(add_noise(self.bn3_0(self.c3_0(h))))
        return self.l4(h)

In [0]:
class DCGANUpdater(chainer.training.StandardUpdater):

    def __init__(self, *args, **kwargs):
        self.gen, self.dis = kwargs.pop('models')
        super(DCGANUpdater, self).__init__(*args, **kwargs)

    def loss_dis(self, dis, y_fake, y_real):
        batchsize = len(y_fake)
        L1 = F.sum(F.softplus(-y_real)) / batchsize
        L2 = F.sum(F.softplus(y_fake)) / batchsize
        loss = L1 + L2
        chainer.report({'loss': loss}, dis)
        return loss

    def loss_gen(self, gen, y_fake):
        batchsize = len(y_fake)
        loss = F.sum(F.softplus(-y_fake)) / batchsize
        chainer.report({'loss': loss}, gen)
        return loss

    def update_core(self):
        gen_optimizer = self.get_optimizer('gen')
        dis_optimizer = self.get_optimizer('dis')
        #get_iterator:与えられた名称の Dataset iterator を取得する
        batch = self.get_iterator('main').next()
        x_real = Variable(self.converter(batch, self.device)) / 255.
        xp = chainer.cuda.get_array_module(x_real.data)

        gen, dis = self.gen, self.dis
        batchsize = len(batch)

        y_real = dis(x_real)

        z = Variable(xp.asarray(gen.make_hidden(batchsize)))
        x_fake = gen(z)
        y_fake = dis(x_fake)

        dis_optimizer.update(self.loss_dis, dis, y_fake, y_real)
        gen_optimizer.update(self.loss_gen, gen, y_fake)

In [15]:
def main4():
    batch_size = 128
    num_classes = 10
    epochs = 1
    chanel = 1
    val_interval = (100 if True else 100000), 'iteration'
    log_interval = (100 if True else 1000), 'iteration'
    train, _ = chainer.datasets.get_cifar10(withlabel=False, scale=255.)
    print(train.shape)
    train_iter = chainer.iterators.SerialIterator(train, batch_size)
    
    chainer.cuda.get_device_from_id(0).use()
    #「モデル作成」
    #ジェネレータ
    gen = Generator(n_hidden=n_hidden)
    #ディスクリメータ
    dis = Discriminator()
    gen.to_gpu()  # Copy the model to the GPU
    dis.to_gpu()
    
    opt_gen = chainer.optimizers.Adam(alpha=0.0002, beta1=0.5)
    opt_gen.setup(gen)
    opt_gen.add_hook(chainer.optimizer.WeightDecay(0.0001), 'hook_dec')
    opt_dis = chainer.optimizers.Adam(alpha=0.0002, beta1=0.5)
    opt_dis.setup(dis)
    opt_dis.add_hook(chainer.optimizer.WeightDecay(0.0001), 'hook_dec')
    updater = DCGANUpdater(
        models=(gen, dis),
        iterator=train_iter,
        optimizer={'gen': opt_gen, 'dis': opt_dis}, device=0)
    
    trainer = training.Trainer(updater, (epochs, 'epoch'), out="./")
    trainer.extend(extensions.LogReport(trigger=log_interval))
    trainer.extend(extensions.PrintReport([
        'epoch', 'iteration', 'gen/loss', 'dis/loss',
    ]), trigger=log_interval)
    trainer.extend(extensions.ProgressBar(update_interval=100))
    trainer.run()
    
    #予測
#     xp = cuda.cupy
#     for x_train_ in train_iter:
#       print(x_train_[0][0].shape)
#       x = x_train_[0][0].reshape(1, 28, 28, 1).transpose(3, 0, 1, 2)
#       print(model.predict(xp.asarray(x)))
#       break

if __name__ == '__main__':
    main4()

epoch       iteration   gen/loss    dis/loss  
0           100         1.37738     1.60811     
     total [############......................................] 25.60%
this epoch [############......................................] 25.60%
       100 iter, 0 epoch / 1 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
0           200         1.16773     1.29288     
     total [#########################.........................] 51.20%
this epoch [#########################.........................] 51.20%
       200 iter, 0 epoch / 1 epochs
    1.3672 iters/sec. Estimated time to finish: 0:02:19.429354.
0           300         1.33162     1.17906     
     total [######################################............] 76.80%
this epoch [######################################............] 76.80%
       300 iter, 0 epoch / 1 epochs
    1.3671 iters/sec. Estimated time to finish: 0:01:06.289364.
